In [ ]:
#| default_exp utils_blog_tests

In [ ]:
from fh_saas.utils_blog import PostLoader, MarkdownEngine, _generate_slug, _parse_date
from pathlib import Path
import pytest
from datetime import datetime

## Helper Function Tests

In [ ]:
#| hide

def test_generate_slug():
    """Test slug generation from filenames"""
    assert _generate_slug('bg0010.md') == 'bg0010'
    assert _generate_slug('My Post.md') == 'my-post'
    assert _generate_slug('Test File 123.md') == 'test-file-123'

In [ ]:
#| hide

def test_parse_date():
    """Test date parsing from various formats"""
    # datetime object
    dt = datetime(2024, 1, 15, 10, 30)
    assert _parse_date(dt) == dt
    
    # ISO string
    date_str = '2024-01-15T10:30:00'
    parsed = _parse_date(date_str)
    assert parsed.year == 2024
    assert parsed.month == 1
    assert parsed.day == 15
    
    # UTC string
    date_utc = '2024-01-15T10:30:00Z'
    parsed_utc = _parse_date(date_utc)
    assert parsed_utc is not None
    
    # None
    assert _parse_date(None) is None
    
    # Invalid string
    assert _parse_date('invalid') is None

## PostLoader Tests

In [ ]:
#| hide

def test_postloader_empty_directory(tmp_path):
    """Test PostLoader with empty directory"""
    loader = PostLoader(tmp_path)
    posts = loader.load_posts()
    assert posts == []

In [ ]:
#| hide

def test_postloader_nonexistent_directory():
    """Test PostLoader with nonexistent directory"""
    loader = PostLoader('/nonexistent/path')
    posts = loader.load_posts()
    assert posts == []

In [ ]:
#| hide

def test_postloader_load_posts(tmp_path):
    """Test loading posts with frontmatter"""
    # Create test markdown files
    post1 = tmp_path / 'post1.md'
    post1.write_text('''---
title: First Post
date: 2024-01-15
categories:
  - tech
  - python
author: Alice
description: First post description
---

# Hello World

This is the first post.
''')
    
    post2 = tmp_path / 'post2.md'
    post2.write_text('''---
title: Second Post
date: 2024-01-20
series: Tutorial Series
---

# Second Post

This is the second post.
''')
    
    loader = PostLoader(tmp_path)
    posts = loader.load_posts()
    
    assert len(posts) == 2
    
    # Check sorting (newest first)
    assert posts[0]['title'] == 'Second Post'
    assert posts[1]['title'] == 'First Post'
    
    # Check first post
    assert posts[1]['slug'] == 'post1'
    assert posts[1]['categories'] == ['tech', 'python']
    assert posts[1]['author'] == 'Alice'
    assert posts[1]['description'] == 'First post description'
    assert '# Hello World' in posts[1]['body']
    
    # Check second post
    assert posts[0]['slug'] == 'post2'
    assert posts[0]['series'] == 'Tutorial Series'
    assert '# Second Post' in posts[0]['body']

In [ ]:
#| hide

def test_postloader_get_post(tmp_path):
    """Test getting single post by slug"""
    post_file = tmp_path / 'my-post.md'
    post_file.write_text('''---
title: My Post
---

Content here.
''')
    
    loader = PostLoader(tmp_path)
    
    # Get existing post
    post = loader.get_post('my-post')
    assert post is not None
    assert post['title'] == 'My Post'
    
    # Get nonexistent post
    missing = loader.get_post('nonexistent')
    assert missing is None

In [ ]:
#| hide

def test_postloader_missing_frontmatter(tmp_path):
    """Test post with no frontmatter uses filename as title"""
    post_file = tmp_path / 'no-frontmatter.md'
    post_file.write_text('# Just Content\n\nNo frontmatter here.')
    
    loader = PostLoader(tmp_path)
    posts = loader.load_posts()
    
    assert len(posts) == 1
    assert posts[0]['title'] == 'no-frontmatter'
    assert posts[0]['slug'] == 'no-frontmatter'
    assert '# Just Content' in posts[0]['body']

## MarkdownEngine Tests

In [ ]:
#| hide

def test_markdown_engine_basic():
    """Test basic markdown rendering"""
    engine = MarkdownEngine()
    html = engine.render('# Hello\n\nThis is **bold** text.')
    
    assert '<h1>Hello</h1>' in html
    assert '<strong>bold</strong>' in html
    assert '<p>' in html

In [ ]:
#| hide

def test_markdown_engine_code_blocks():
    """Test fenced code blocks with syntax highlighting"""
    engine = MarkdownEngine()
    html = engine.render('''```python
def hello():
    print("world")
```''')
    
    assert 'class="highlight"' in html
    assert 'def hello' in html

In [ ]:
#| hide

def test_markdown_engine_tables():
    """Test markdown table rendering"""
    engine = MarkdownEngine()
    html = engine.render('''| Name | Age |
| --- | --- |
| Alice | 30 |
| Bob | 25 |''')
    
    assert '<table>' in html
    assert '<th>Name</th>' in html
    assert '<td>Alice</td>' in html

In [ ]:
#| hide

def test_markdown_engine_toc():
    """Test table of contents generation"""
    engine = MarkdownEngine()
    html = engine.render('''# Main Title

## Section 1

Content here.

## Section 2

More content.''')
    
    toc = engine.get_toc()
    
    assert toc != ''
    assert 'Section 1' in toc
    assert 'Section 2' in toc
    assert '<a href="#section-1">' in toc or 'section-1' in toc

In [ ]:
#| hide

def test_markdown_engine_reset():
    """Test that engine resets state between renders"""
    engine = MarkdownEngine()
    
    # First render with TOC
    html1 = engine.render('## Heading 1')
    toc1 = engine.get_toc()
    assert 'Heading 1' in toc1
    
    # Second render without headings
    html2 = engine.render('Just text, no headings.')
    toc2 = engine.get_toc()
    
    # TOC should be empty or not contain old heading
    assert 'Heading 1' not in toc2